Dependencies

In [2]:
#dependencies
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.optimizers import RMSprop, Adam
import gym


Using TensorFlow backend.


Q(state, action) = R(state, action) + Gamma * Max[Q(next state, all actions)]

The Gamma parameter has a range of 0 to 1 (0 <= Gamma > 1).  If Gamma is closer to zero, the agent will tend to consider only immediate rewards.  If Gamma is closer to one, the agent will consider future rewards with greater weight, willing to delay the reward.

epsilon parameter determines with which probability our agent takes a random action

## First model
first model 

In [5]:
class Agent:
    def __init__(self, envName=('CartPole-v0'), gamma=None, epsilon=None, e_decay=0.995, learning_rate=0.001):
        self.envName = envName
        self.env = gym.make(envName)
        self.memory = deque(maxlen=10000)
        self.gamma = gamma
        self.epsilon = epsilon
        self.e_decay = e_decay
        self.e_min = 0.02  # minimal exploraton rate
        self.learning_rate = learning_rate
        self.model = self._build_model()

    def _build_model(self):
        '''
        builds a neural network
        :return: model - neural network object
        '''
        model = Sequential()
        model.add(Dense(24, input_dim=self.env.observation_space.shape[0], activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.env.action_space.n, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        '''
        add <s,a,r,s'> vector to memory
        :param state: agent state
        :param action: action to perform
        :param reward: reward given after performed action
        :param next_state: next state
        :param done: bool parameter which determines whrther current state terminal or not
        :return: 
        '''
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        '''
        defines gow agent will actin a given state
        :param state: state of our agent
        :return: action to be perfomed
        '''
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.env.action_space.n)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        '''
        train network by sampling elements from memory
        :param batch_size: sample size of agent memory
        :return: 
        '''
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        X = np.zeros((batch_size, self.env.observation_space.shape[0]))
        Y = np.zeros((batch_size, self.env.action_space.n))
        for i in range(batch_size):
            state, action, reward, next_state, done = minibatch[i]
            target = self.model.predict(state)[0]
            # print "i", i, " predict", self.model.predict(state)
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            X[i], Y[i] = state, target
        self.model.fit(X, Y, batch_size=batch_size, nb_epoch=1, verbose=0)
        if self.epsilon > self.e_min:
            self.epsilon *= self.e_decay

    def load(self, name):
        '''
        load weights from a file
        :param name: filename
        :return: 
        '''
        self.model.load_weights(name)

    def save(self, name):
        '''
        seves weights to a file
        :param name: filename
        :return: 
        '''
        self.model.save_weights(name)


### Cartpole

In [13]:
agent = Agent('CartPole-v0',.95,1.0)
# Iterate the game
#####################################
#uncomment if u want to load weights#
#####################################
# agent.model.load_weights("cartpole3.kek")

#####################################
#uncomment if u want to save scores#
#####################################
f = open('data/CPscores.txt', 'w')

for e in range(2000):
    state = agent.env.reset()
    state_size = agent.env.observation_space.shape[0]
    state = np.reshape(state, [1,state_size])
    resultReward = 0
    while True:
        agent.env.render()
        action = agent.act(state)
        next_state, reward, done, _ = agent.env.step(action)
        # reward = reward if not done else -1
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        resultReward += reward
        if done and resultReward <= 190:
            reward = -1
        else:
            reward = reward
        if done:
            if resultReward > 180:
                print 'episode ', e, ' score: ', resultReward
            print >> f, resultReward #, 'ep', e
            break
    agent.replay(64)
    if e % 10 == 0:
         agent.save("cartpole2.kek")
f.close()

[2017-10-17 13:20:00,571] Making new env: CartPole-v0


episode  718  score:  200.0
episode  719  score:  200.0
episode  722  score:  200.0
episode  723  score:  200.0
episode  725  score:  200.0
episode  972  score:  200.0
episode  977  score:  200.0
episode  994  score:  200.0
episode  1024  score:  200.0
episode  1057  score:  197.0
episode  1085  score:  200.0
episode  1086  score:  200.0
episode  1087  score:  200.0
episode  1088  score:  200.0
episode  1089  score:  200.0
episode  1090  score:  200.0
episode  1093  score:  200.0
episode  1094  score:  200.0
episode  1095  score:  200.0
episode  1146  score:  182.0
episode  1147  score:  200.0
episode  1152  score:  195.0
episode  1156  score:  200.0
episode  1163  score:  200.0
episode  1164  score:  200.0
episode  1169  score:  200.0
episode  1197  score:  200.0
episode  1198  score:  200.0
episode  1200  score:  200.0
episode  1202  score:  200.0
episode  1204  score:  196.0
episode  1205  score:  200.0
episode  1214  score:  200.0
episode  1240  score:  200.0
episode  1241  score: 

### Mountain Car

In [14]:
agent = Agent('MountainCar-v0',1.,1.0,0.9995,0.0001)
# Iterate the game
f=open('mountainCarFirstModel.txt','w')
for e in range(10000):
    state = agent.env.reset()
    state_size = agent.env.observation_space.shape[0]
    state = np.reshape(state, [1,state_size])
    resultReward = 0
    while True:
        agent.env.render()
        action = agent.act(state)
        next_state, reward, done, _ = agent.env.step(action)
        # reward = reward if not done else -1
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        resultReward += reward
        if done:
            if resultReward > -200:
                print 'ep', e , 'score', resultReward
            print >> f, resultReward
            break
    agent.replay(64)

    if e % 50 == 0:
        agent.save("MountainCarFirstModelWeights.kek")
f.close()


[2017-10-17 13:31:50,313] Making new env: MountainCar-v0


# Second model

In [16]:
class Agent2:
    def __init__(self, state_size, action_size):
        self.render = True

        self.state_size = state_size
        self.action_size = action_size

        self.memory = deque(maxlen=10000) #memory size
        self.discount_factor = 0.99
        self.epsilon = 1.0
        self.e_min = 0.005
        self.e_decay = 0.00002
        self.train_start = 1000

        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        '''
        build neural net
        :return: model - neural net object
        '''
        model = Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear', kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=0.001))
        return model

    def update_target_model(self):
        '''
        update weights of our target model
        :return:
        '''
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        '''
        perform agent action in a given state
        :param state: agent state
        :return:
        '''
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    def remember(self, state, action, reward, next_state, done):
        '''
        add <s,a,r,s'> vector to agent memory
        :param state: agent state
        :param action: action to perform
        :param reward: collected reward
        :param next_state: next state after performing actopn
        :param done: bool variable which determines is current state terminal or not
        :return:
        '''
        if action == 2:
            action = 1
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.e_min:
            self.epsilon -= self.e_decay
        # print(len(self.memory))

    def replay(self, batch_size):
        '''
        train network by sampling elements from memory
        :param batch_size:
        :return:
        '''
        if len(self.memory) < self.train_start:
            return
        batch_size = min(batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.action_size))

        for i in range(batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            target = self.model.predict(state)[0]

            if done:
                target[action] = reward
            else:
                target[action] = reward + self.discount_factor * \
                                          np.amax(self.target_model.predict(next_state)[0])
            update_input[i] = state
            update_target[i] = target

        self.model.fit(update_input, update_target, batch_size=batch_size, epochs=1, verbose=0)

    def load(self, name):
        '''
        loadweights of pre-trained network
        :param name: filename to load
        :return:
        '''
        self.model.load_weights(name)

    def save(self, name):
        '''
        store network weights
        :param name: filename to save
        :return:
        '''
        self.model.save_weights(name)



### Cartpole

In [19]:
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
#action_size = env.action_space.n
# reduce action size for mountain car problem
action_size = 2
agent = Agent2(state_size, action_size)
# agent.load("./save/MountainCar1.kek")
# uncomment to save scores to a file
f= open('scoresCartpoleSecondModel.txt', 'w')
for e in range(3000):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    # print(state)


    while not done:
        if agent.render:
            env.render()
        action = agent.act(state)

        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        #reward = reward if not done else -100

        agent.remember(state, action, reward, next_state, done)
        score += reward
        state = next_state

        if done:
            env.reset()
            agent.update_target_model()
            # print ony successful episodes
            if score > 180:
                print "episode:", e, "  score:", score
            # uncomment to save scores to a file
            print >> f, score
    agent.replay(64)

    # save model weights every 30 iterations
    if e % 10 == 0:
        agent.save("cartpolesecondModel.kek")
# file close, uncomment if you want to use file
f.close()

[2017-10-17 23:10:14,265] Making new env: CartPole-v0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_41 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 34        
Total params: 722
Trainable params: 722
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_44 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_45 (De

### mountain car

In [ ]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
#action_size = env.action_space.n
# reduce action size for mountain car problem
action_size = 2
agent = Agent2(state_size, action_size)
# agent.load("./save/MountainCar1.kek")
# uncomment to save scores to a file
#f= open('scoresCar1.txt', 'w')
for e in range(100000):
    done = False
    score = 0
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    # print(state)

    take_action = 0

    action_count = 0

    while not done:
        if agent.render:
            env.render()

        action_count = action_count + 1

        if action_count == 4:
            action = agent.act(state)
            action_count = 0

            if action == 0:
                take_action = 0
            elif action == 1:
                take_action = 2

        next_state, reward, done, info = env.step(take_action)
        next_state = np.reshape(next_state, [1, state_size])
        #reward = reward if not done else -100

        agent.remember(state, take_action, reward, next_state, done)
        agent.replay(64)
        score += reward
        state = next_state

        if done:
            env.reset()
            agent.update_target_model()
            # print ony successful episodes
            if score > -200:
                print "episode:", e, "  score:", score
            # uncomment to save scores to a file
            #print >> f,"episode:", e, "  score:", score
    # save model weights every 50 iterations
    if e % 50 == 0:
        agent.save("MountainCar1.kek")
# file close, uncomment if you want to use file
# f.close()

# Results